
#### Creating some sample views whose source is gold layer's user and event data

NOTE: Earlier used CTEs but if reusablity is concerned, re-using TEMP views is faster and easier to backtrack things

In [0]:
%sql
-- Ensure prerequisites (run once)
USE CATALOG inshort_cata;
USE SCHEMA gold;

-- Convert timestamps and handle nulls
CREATE OR REPLACE TEMPORARY VIEW user_clean AS
SELECT 
    deviceid,
    lang,
    COALESCE(district, 'N/A') as district,
    platform,
    to_date(install_dt) as install_date,
    campaign_id
FROM users;

CREATE OR REPLACE TEMPORARY VIEW event_clean AS
SELECT 
    deviceid,
    content_id,
    eventtimestamp,
    timespent,
    eventname,
    to_date(from_unixtime(eventtimestamp/1000)) as event_date
FROM events 
WHERE eventname IN ('Opened', 'Shown', 'Shared');


-- Creating user cohort by their install date
CREATE OR REPLACE TEMPORARY VIEW user_cohorts AS
SELECT 
    deviceid,
    DATE_TRUNC('day', install_date) as cohort_date,
    install_date
FROM user_clean;

#### Retention Metrics:

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW user_cohorts AS
SELECT 
    deviceid,
    DATE_TRUNC('day', install_date) as cohort_date,
    install_date
FROM user_clean;

CREATE OR REPLACE TEMPORARY VIEW user_activity AS
SELECT 
    uc.deviceid,
    uc.cohort_date,
    uc.install_date,
    MIN(ec.event_date) as first_activity_date,
    COUNT(*) as total_sessions
FROM user_cohorts uc
LEFT JOIN event_clean ec ON uc.deviceid = ec.deviceid 
    AND ec.eventname = 'Opened'
GROUP BY uc.deviceid, uc.cohort_date, uc.install_date;


In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW retention_cohort AS
SELECT 
    cohort_date,
    COUNT(*) as cohort_size,
    -- D1: Day 1 active users
    COUNT(CASE WHEN DATEDIFF(first_activity_date, install_date) = 1 THEN 1 END) as d1_active,
    -- D2: Day 2 active users  
    COUNT(CASE WHEN DATEDIFF(first_activity_date, install_date) = 2 THEN 1 END) as d2_active,
    -- W1: Week 1 (days 1-7)
    COUNT(CASE WHEN DATEDIFF(first_activity_date, install_date) BETWEEN 1 AND 7 THEN 1 END) as w1_active,
    -- M1: Month 1 (days 1-30)
    COUNT(CASE WHEN DATEDIFF(first_activity_date, install_date) BETWEEN 1 AND 30 THEN 1 END) as m1_active,
    -- M3: Month 3 (days 1-90)
    COUNT(CASE WHEN DATEDIFF(first_activity_date, install_date) BETWEEN 1 AND 90 THEN 1 END) as m3_active
FROM user_activity
GROUP BY cohort_date;

In [0]:
%sql
SELECT 
    cohort_date,
    cohort_size,
    ROUND((d1_active * 100.0 / cohort_size), 2) as d1_retention_pct,
    ROUND((d2_active * 100.0 / cohort_size), 2) as d2_retention_pct,
    ROUND((w1_active * 100.0 / cohort_size), 2) as w1_retention_pct,
    ROUND((m1_active * 100.0 / cohort_size), 2) as m1_retention_pct,
    ROUND((m3_active * 100.0 / cohort_size), 2) as m3_retention_pct
FROM retention_cohort 
ORDER BY cohort_date

cohort_date,cohort_size,d1_retention_pct,d2_retention_pct,w1_retention_pct,m1_retention_pct,m3_retention_pct
2024-01-01T00:00:00.000Z,1327,16.43,6.78,30.82,36.77,38.73
2024-01-02T00:00:00.000Z,1753,13.23,4.91,26.64,33.26,34.91
2024-01-03T00:00:00.000Z,1562,11.46,5.95,26.63,34.76,36.62
2024-01-04T00:00:00.000Z,1321,16.20,4.09,28.99,37.70,39.59
2024-01-05T00:00:00.000Z,1428,11.27,5.67,26.05,33.47,35.99
2024-01-06T00:00:00.000Z,1256,12.10,5.10,26.27,36.15,38.54
2024-01-07T00:00:00.000Z,1333,12.83,5.40,29.86,36.61,38.33
2024-01-08T00:00:00.000Z,1468,13.90,6.13,29.50,36.24,38.22
2024-01-09T00:00:00.000Z,1482,15.52,5.67,29.76,36.64,38.53
2024-01-10T00:00:00.000Z,1437,15.03,5.36,30.41,38.20,40.85


##### Weekly churn:

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW weekly_churn AS
SELECT 
    DATE_TRUNC('week', cohort_date) as cohort_week,
    COUNT(CASE WHEN DATEDIFF(first_activity_date, install_date) BETWEEN 1 AND 7 THEN 1 END) as week1_active,
    COUNT(CASE WHEN DATEDIFF(first_activity_date, install_date) BETWEEN 8 AND 14 THEN 1 END) as week2_active,
    COUNT(*) as cohort_size
FROM user_activity
GROUP BY 1;

SELECT 
    cohort_week,
    cohort_size,
    ROUND((week1_active * 100.0 / cohort_size), 2) as week1_retention,
    ROUND((week2_active * 100.0 / cohort_size), 2) as week2_retention,
    ROUND(((week1_active - week2_active) * 100.0 / week1_active), 2) as weekly_churn_pct
FROM weekly_churn
ORDER BY cohort_week;

cohort_week,cohort_size,week1_retention,week2_retention,weekly_churn_pct
2024-01-01T00:00:00.000Z,9980,27.81,4.27,84.65
2024-01-08T00:00:00.000Z,9224,29.38,4.25,85.54
2024-01-15T00:00:00.000Z,8786,29.69,3.74,87.39
2024-01-22T00:00:00.000Z,9668,28.66,3.64,87.30
2024-01-29T00:00:00.000Z,4850,28.31,3.77,86.67


##### Monthly churn by District and Platform:

In [0]:
%sql
SELECT 
    uc.district,
    uc.platform,
    COUNT(DISTINCT ua.deviceid) as total_users,
    COUNT(CASE WHEN DATEDIFF(ua.first_activity_date, uc.install_date) <= 30 THEN 1 END) as m1_active,
    COUNT(CASE WHEN DATEDIFF(ua.first_activity_date, uc.install_date) > 30 THEN 1 END) as m2_churned,
    ROUND(try_divide(m1_active * 100.0 , total_users), 2) as m1_retention_pct,
    ROUND(try_divide(m2_churned * 100.0 , m1_active), 2) as monthly_churn_pct
FROM user_clean uc
JOIN user_activity ua ON uc.deviceid = ua.deviceid
WHERE uc.district != 'N/A'
GROUP BY uc.district, uc.platform
ORDER BY monthly_churn_pct DESC
;

district,platform,total_users,m1_active,m2_churned,m1_retention_pct,monthly_churn_pct
MH_JN,ANDROID,2,1,1,50.00,100.00
JH_GU,ANDROID,2,1,1,50.00,100.00
HR_AM,ANDROID,5,1,1,20.00,100.00
AP_KU,ANDROID,2,1,1,50.00,100.00
UP_FR,ANDROID,8,2,1,25.00,50.00
MP_HO,ANDROID,6,2,1,33.33,50.00
GJ_BR,ANDROID,6,2,1,33.33,50.00
OD_PU,ANDROID,4,2,1,50.00,50.00
WB_HG,ANDROID,5,2,1,40.00,50.00
LK_LE,ANDROID,3,2,1,66.67,50.00


##### Churn by Platform: 

In [0]:
%sql
SELECT 
    uc.platform,
    COUNT(*) as total_users,
    COUNT(CASE WHEN DATEDIFF(ua.first_activity_date, uc.install_date) <= 30 THEN 1 END) as m1_active,
    ROUND((COUNT(CASE WHEN DATEDIFF(ua.first_activity_date, uc.install_date) <= 30 THEN 1 END) * 100.0 / COUNT(*)), 2) as m1_retention_pct,
    ROUND((COUNT(CASE WHEN DATEDIFF(ua.first_activity_date, uc.install_date) > 30 THEN 1 END) * 100.0 / 
           COUNT(CASE WHEN DATEDIFF(ua.first_activity_date, uc.install_date) <= 30 THEN 1 END)), 2) as monthly_churn_pct
FROM user_clean uc
JOIN user_activity ua ON uc.deviceid = ua.deviceid
GROUP BY uc.platform
ORDER BY monthly_churn_pct DESC;

platform,total_users,m1_active,m1_retention_pct,monthly_churn_pct
ANDROID,34842,19120,54.88,4.24
IOS,7666,3428,44.72,4.23


##### Churn by Language:

In [0]:
%sql
SELECT 
    uc.lang,
    COUNT(*) as total_users,
    ROUND((COUNT(CASE WHEN DATEDIFF(ua.first_activity_date, uc.install_date) <= 30 THEN 1 END) * 100.0 / COUNT(*)), 2) as m1_retention_pct,
    ROUND((COUNT(CASE WHEN DATEDIFF(ua.first_activity_date, uc.install_date) > 30 THEN 1 END) * 100.0 / 
           COUNT(CASE WHEN DATEDIFF(ua.first_activity_date, uc.install_date) <= 30 THEN 1 END)), 2) as monthly_churn_pct
FROM user_clean uc
JOIN user_activity ua ON uc.deviceid = ua.deviceid
GROUP BY uc.lang
ORDER BY monthly_churn_pct DESC;

lang,total_users,m1_retention_pct,monthly_churn_pct
hi,6707,54.05,4.52
en,35801,52.86,4.18


##### Churn risks:

In [0]:
%sql
SELECT 
    r.district,
    r.platform,
    r.monthly_churn_pct
FROM (
    SELECT 
        uc.district,
        uc.platform,
        ROUND(
            try_divide(
                COUNT(CASE WHEN DATEDIFF(ua.first_activity_date, uc.install_date) > 30 THEN 1 END) * 100.0,
                COUNT(CASE WHEN DATEDIFF(ua.first_activity_date, uc.install_date) <= 30 THEN 1 END)
            ), 2
        ) as monthly_churn_pct,
        ROW_NUMBER() OVER (
            ORDER BY 
                ROUND(
                    try_divide(
                        COUNT(CASE WHEN DATEDIFF(ua.first_activity_date, uc.install_date) > 30 THEN 1 END) * 100.0,
                        COUNT(CASE WHEN DATEDIFF(ua.first_activity_date, uc.install_date) <= 30 THEN 1 END)
                    ), 2
                ) DESC
        ) as rn
    FROM user_clean uc
    JOIN user_activity ua ON uc.deviceid = ua.deviceid
    WHERE uc.district != 'N/A'
    GROUP BY uc.district, uc.platform
) r
-- WHERE r.rn <= 5  -- considering only top 5 for now, optional filter
ORDER BY r.monthly_churn_pct DESC;


district,platform,monthly_churn_pct
MH_JN,ANDROID,100.00
JH_GU,ANDROID,100.00
HR_AM,ANDROID,100.00
AP_KU,ANDROID,100.00
UP_FR,ANDROID,50.00
MP_HO,ANDROID,50.00
GJ_BR,ANDROID,50.00
OD_PU,ANDROID,50.00
WB_HG,ANDROID,50.00
LK_LE,ANDROID,50.00
